<a href="https://colab.research.google.com/github/shubhamphal/One-NLP/blob/master/Detecting_Contradiction_and_Entailment_in_Multi_lingual_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_text

     |████████████████████████████████| 2.6MB 7.0MB/s 
     |████████████████████████████████| 320.4MB 55kB/s 
     |████████████████████████████████| 460kB 31.4MB/s 
     |████████████████████████████████| 6.8MB 41.7MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [ ]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.3.0)


In [ ]:
 !pip install -q kaggle

In [ ]:
 !mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c contradictory-my-dear-watson

  0% 0.00/1.23M [00:00<?, ?B/s]
100% 1.23M/1.23M [00:00<00:00, 39.8MB/s]
  0% 0.00/536k [00:00<?, ?B/s]
100% 536k/536k [00:00<00:00, 174MB/s]
  0% 0.00/66.0k [00:00<?, ?B/s]
100% 66.0k/66.0k [00:00<00:00, 69.1MB/s]


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow_text
import os
import sys
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, GlobalMaxPooling1D, Bidirectional, LSTM, GRU, GlobalMaxPool1D, Dropout, Lambda, Concatenate
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from sklearn.metrics import roc_auc_score
from nltk.corpus import stopwords
from keras.optimizers import Adam
from nltk.tokenize import word_tokenize


In [ ]:
#Extract test, train and labels 
import zipfile
with zipfile.ZipFile('test.csv.zip', 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())
with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())

In [ ]:
train_data=pd.read_csv('train.csv')

In [ ]:
train_data['language'].unique()

array(['English', 'French', 'Thai', 'Turkish', 'Urdu', 'Russian',
       'Bulgarian', 'German', 'Arabic', 'Chinese', 'Hindi', 'Swahili',
       'Vietnamese', 'Spanish', 'Greek'], dtype=object)

In [ ]:
universal_sentence_encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [ ]:
train_data.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [ ]:
premise_sentences=train_data.premise

In [ ]:
hypothesis_sentences=train_data.hypothesis

In [ ]:
X_train_premise = []
for r in tqdm(premise_sentences):
  emb = universal_sentence_encoder(r)
  premise_emb = tf.reshape(emb, [-1,1]).numpy()
  X_train_premise.append(premise_emb)


100%|██████████| 12120/12120 [05:48<00:00, 34.76it/s]


In [ ]:
X_train_premise = np.array(X_train_premise)

In [ ]:
len(X_train_premise)

12120

In [ ]:
X_train_hypothesis = []
for r in tqdm(hypothesis_sentences):
  emb = universal_sentence_encoder(r)
  hypothesis_emb = tf.reshape(emb, [-1,1]).numpy()
  X_train_hypothesis.append(hypothesis_emb)

100%|██████████| 12120/12120 [05:21<00:00, 37.74it/s]


In [ ]:
X_train_hypothesis=np.array(X_train_hypothesis)

In [ ]:
len(X_train_hypothesis)

12120

In [ ]:
X_train_hypothesis[0].shape

(512, 1)

In [ ]:
def format_target(target):
  if target==0:
    return 'entailment'
  elif target==1:
    return 'neutral'
  else:
    return 'contradiction'

In [ ]:
train_data['formatted_labels']=train_data['label'].apply(format_target)

In [ ]:
train_one_hot_data=pd.get_dummies(train_data,columns=['formatted_labels'])

In [ ]:
train_one_hot_data.head()

,id,premise,hypothesis,lang_abv,language,label,formatted_labels_contradiction,formatted_labels_entailment,formatted_labels_neutral
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0,0,1,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2,1,0,0
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0,0,1,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0,0,1,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1,0,0,1


In [ ]:
train_one_hot_data['contradiction']=train_one_hot_data['formatted_labels_contradiction']
train_one_hot_data['entailment']=train_one_hot_data['formatted_labels_entailment']
train_one_hot_data['neutral']=train_one_hot_data['formatted_labels_neutral']

In [ ]:
train_one_hot_data=train_one_hot_data.drop(columns=['formatted_labels_contradiction','formatted_labels_entailment','formatted_labels_neutral'],axis=1)

In [ ]:
train_one_hot_data.head()

,id,premise,hypothesis,lang_abv,language,label,contradiction,entailment,neutral
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0,0,1,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2,1,0,0
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0,0,1,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0,0,1,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1,0,0,1


In [ ]:
possible_labels=['entailment','neutral','contradiction']

In [ ]:
targets=train_one_hot_data[possible_labels].values

In [ ]:
MAX_SEQUENCE_LENGTH=512

In [ ]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [ ]:
def do_nothing(x):
  return x


In [ ]:
embedding = layers.Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)

In [ ]:
embedding_layer=Embedding(num_words,EMBEDDING_DIM,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=False)

In [ ]:
X_train_premise.shape

(12120, 512, 1)

In [ ]:
inputA = Input(shape=(MAX_SEQUENCE_LENGTH,1))
inputB = Input(shape=(MAX_SEQUENCE_LENGTH,1))

rnn1 = Bidirectional(GRU(64, return_sequences=True))
x1 = rnn1(inputA) 
x1 = GlobalMaxPooling1D()(x1) 

rnn2 = Bidirectional(GRU(64, return_sequences=True))
x2 = rnn2(inputB) 
x2 = GlobalMaxPooling1D()(x2)

concatenator = Concatenate(axis=1)
x = concatenator([x1, x2])

x=Dense(256,activation='relu')(x)
x=Dense(64,activation='relu')(x)
x=Dense(256,activation='relu')(x)

output_ = Dense(3, activation='softmax')(x)

In [ ]:
model = Model(inputs=[inputA,inputB], outputs=output_)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
r=model.fit([X_train_premise,X_train_hypothesis],targets,epochs=10,validation_split=0.1,batch_size=32)

Epoch 1/10
341/341 [==============================] - 129s 379ms/step - loss: 1.0987 - accuracy: 0.3402 - val_loss: 1.0980 - val_accuracy: 0.3383
Epoch 2/10
341/341 [==============================] - 128s 376ms/step - loss: 1.0984 - accuracy: 0.3453 - val_loss: 1.0984 - val_accuracy: 0.3383
Epoch 3/10
341/341 [==============================] - 128s 376ms/step - loss: 1.0985 - accuracy: 0.3398 - val_loss: 1.0979 - val_accuracy: 0.3383
Epoch 4/10
341/341 [==============================] - 128s 376ms/step - loss: 1.0984 - accuracy: 0.3453 - val_loss: 1.0980 - val_accuracy: 0.3383
Epoch 5/10
341/341 [==============================] - 128s 376ms/step - loss: 1.0984 - accuracy: 0.3453 - val_loss: 1.0979 - val_accuracy: 0.3383
Epoch 6/10
341/341 [==============================] - 128s 376ms/step - loss: 1.0984 - accuracy: 0.3453 - val_loss: 1.0981 - val_accuracy: 0.3383
Epoch 7/10
341/341 [==============================] - 128s 376ms/step - loss: 1.0984 - accuracy: 0.3453 - val_loss: 1.0978 -

In [ ]:
test_data=pd.read_csv('test.csv')

In [ ]:
test_data.head()

,id,premise,hypothesis,lang_abv,language
0,c6d58c3f69,بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...,"کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...",ur,Urdu
1,cefcc82292,هذا هو ما تم نصحنا به.,عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...,ar,Arabic
2,e98005252c,et cela est en grande partie dû au fait que le...,Les mères se droguent.,fr,French
3,58518c10ba,与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp,IMA与其他组织合作，因为它们都依靠共享资金。,zh,Chinese
4,c32b0d16df,Она все еще была там.,"Мы думали, что она ушла, однако, она осталась.",ru,Russian


In [ ]:
test_premise_sentences=test_data.premise.values

In [ ]:
test_hypothesis_sentences=test_data.hypothesis.values

In [ ]:
X_test_premise = []
for r in tqdm(test_premise_sentences):
  emb = universal_sentence_encoder(r)
  premise_emb = tf.reshape(emb, [-1,1]).numpy()
  X_test_premise.append(premise_emb)

100%|██████████| 5195/5195 [02:32<00:00, 34.00it/s]


In [ ]:
X_test_hypothesis = []
for r in tqdm(test_hypothesis_sentences):
  emb = universal_sentence_encoder(r)
  hypothesis_emb = tf.reshape(emb, [-1,1]).numpy()
  X_test_hypothesis.append(hypothesis_emb)

100%|██████████| 5195/5195 [02:23<00:00, 36.19it/s]


In [ ]:
len(X_test_hypothesis)

5195

In [ ]:
X_test_premise=np.array(X_test_premise)

In [ ]:
X_test_hypothesis=np.array(X_test_hypothesis)

In [ ]:
X_test_hypothesis[0].shape

(512, 1)

In [ ]:
predictions=model.predict([X_test_premise,X_test_hypothesis])

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['prediction'] = np.argmax(predictions,axis=1)
submission.head()

,id,prediction
0,c6d58c3f69,0
1,cefcc82292,0
2,e98005252c,0
3,58518c10ba,0
4,c32b0d16df,0


In [ ]:
submission['prediction'].sum()

0